<a href="https://colab.research.google.com/github/zeroplam5/2025_aiku_junirothon/blob/main/%EC%A0%95%EC%84%B1%EC%9C%A4notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [39]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

aikuthon9th_path = kagglehub.competition_download('aikuthon9th')

print('Data source import complete.')


100%|██████████| 4.04G/4.04G [03:11<00:00, 22.7MB/s]

Extracting files...


Data source import complete.


In [3]:
print(aikuthon9th_path)

/root/.cache/kagglehub/competitions/aikuthon9th


In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(aikuthon9th_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/root/.cache/kagglehub/competitions/aikuthon9th/train_metadata.csv
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/07387.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/02666.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/00459.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/07568.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/01176.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/05807.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/06352.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/07516.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/09660.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/02312.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/00685.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/07097.png
/root/.cache/kagglehub/competitions/aikuthon9th/train_images/06333.png
/root/.cac

In [4]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [5]:
class AikuthonDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, f"{self.dataframe.iloc[idx, 0]:05d}.png")
        image = Image.open(img_name).convert('RGB')
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
# Load the metadata CSV file
metadata_path = os.path.join(aikuthon9th_path, 'train_metadata.csv')
labels_df = pd.read_csv(metadata_path)

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(labels_df, test_size=0.2, random_state=42, stratify=labels_df['label'])

# Display the first few rows of the training and validation dataframes
print("Training DataFrame:")
display(train_df.head())
print("\nValidation DataFrame:")
display(val_df.head())

Training DataFrame:


,id,label
5373,5374,5
450,451,0
2149,2150,0
7221,7222,0
3045,3046,0



Validation DataFrame:


,id,label
4902,4903,0
4840,4841,4
6408,6409,4
2198,2199,0
5850,5851,0


In [7]:
train_img_dir = os.path.join(aikuthon9th_path, 'train_images')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = AikuthonDataset(train_df, train_img_dir, transform=transform)
val_dataset = AikuthonDataset(val_df, train_img_dir, transform=transform)

In [9]:
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [10]:
import torchvision.models as models

# Load a pre-trained ViT-Base/16 model
vit_model = models.vit_b_16(pretrained=True)

print("Pre-trained ViT model loaded.")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:01<00:00, 178MB/s]


Pre-trained ViT model loaded.


In [13]:
# Get the number of input features for the classifier
num_ftrs = vit_model.heads.head.in_features

# Assuming 6 classes for this dataset based on the labels
num_classes = 6

# Replace the pre-trained head with a new one
vit_model.heads.head = torch.nn.Linear(num_ftrs, num_classes)

print(f"ViT model's classification head modified for {num_classes} classes.")

ViT model's classification head modified for 6 classes.


In [12]:
import torch.nn as nn
import torch.optim as optim

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(vit_model.parameters(), lr=0.001)

print("Loss function and optimizer defined.")

Loss function and optimizer defined.


In [11]:
try:
    from tqdm import tqdm
except ImportError:
    print("tqdm not found. Installing...")
    !pip install tqdm
    from tqdm import tqdm

In [14]:
import torchvision.models as models

# Load a pre-trained EfficientNet-B0 model (you can choose a different variant)

efficientnet_model = models.efficientnet_b0(pretrained=True)

print("Pre-trained EfficientNet model loaded.")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 172MB/s]

Pre-trained EfficientNet model loaded.


In [15]:
# Get the number of input features for the classifier
num_ftrs_efficientnet = efficientnet_model.classifier[1].in_features

# Assuming 6 classes for this dataset
num_classes = 6

# Replace the pre-trained classifier with a new one
efficientnet_model.classifier[1] = torch.nn.Linear(num_ftrs_efficientnet, num_classes)

print(f"EfficientNet model's classification head modified for {num_classes} classes.")

EfficientNet model's classification head modified for 6 classes.


In [16]:
import torch.nn as nn
import torch.optim as optim

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
# Use efficientnet_model.parameters() instead of vit_model.parameters()
optimizer = optim.Adam(efficientnet_model.parameters(), lr=0.001)

print("Loss function and optimizer defined for EfficientNet.")

Loss function and optimizer defined for EfficientNet.


In [35]:
import torch.nn as nn
import copy
import time
from sklearn.metrics import f1_score # Import f1_score
import torch # Import torch
from tqdm import tqdm # Import tqdm

class Trainer:
    def __init__(self, model, criterion, optimizer, device='cuda',
                 scheduler=None, mixed_precision=True, patience=5, min_delta=0.001):
        self.model = model.to(device)
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.scaler = torch.cuda.amp.GradScaler(enabled=mixed_precision)
        self.mixed_precision = mixed_precision

        self.best_f1 = 0.0 # Track best F1 score
        self.best_model_wts = copy.deepcopy(self.model.state_dict())
        self.history = {"train_loss": [], "val_loss": [], "train_f1": [], "val_f1": []} # Update history keys

        # Early Stopping parameters
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0 # Counter for early stopping
        self.early_stop = False # Flag for early stopping

    def _run_epoch(self, dataloader, phase='train'):
        if phase == 'train':
            self.model.train()
        else:
            self.model.eval()

        running_loss, total = 0.0, 0
        all_labels = []
        all_predictions = []

        # Apply tqdm to the dataloader iteration
        for inputs, labels in tqdm(dataloader, desc=f"Epoch {self.epoch+1} [{phase}]"):
            inputs = inputs.to(self.device, non_blocking=True)
            labels = labels.to(self.device, non_blocking=True)

            self.optimizer.zero_grad(set_to_none=True)

            with torch.set_grad_enabled(phase == 'train'):
                with torch.autocast(device_type='cuda', dtype=torch.float16, enabled=self.mixed_precision):
                    outputs = self.model(inputs)
                    # Extract logits from the model output
                    logits = outputs.logits if hasattr(outputs, 'logits') else outputs
                    loss = self.criterion(logits, labels)
                _, preds = torch.max(logits, 1) # Use logits for predictions

                if phase == 'train':
                    self.scaler.scale(loss).backward()
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                    if self.scheduler is not None:
                        self.scheduler.step()

            running_loss += loss.item() * inputs.size(0)
            total += inputs.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(preds.cpu().numpy())

        epoch_loss = running_loss / total
        epoch_f1 = f1_score(all_labels, all_predictions, average='weighted') # Calculate F1 score

        return epoch_loss, epoch_f1

    def fit(self, train_loader, val_loader, num_epochs=10):
        for epoch in range(num_epochs):
            self.epoch = epoch # Update current epoch for tqdm desc
            print(f"Epoch {epoch+1}/{num_epochs}")
            print("-" * 30)

            train_loss, train_f1 = self._run_epoch(train_loader, phase='train') # Get train loss and F1
            val_loss, val_f1 = self._run_epoch(val_loader, phase='val') # Get val loss and F1

            self.history["train_loss"].append(train_loss)
            self.history["train_f1"].append(train_f1) # Append train F1
            self.history["val_loss"].append(val_loss)
            self.history["val_f1"].append(val_f1) # Append val F1


            print(f"Train Loss: {train_loss:.4f}, F1 Score: {train_f1:.4f}") # Update print
            print(f"Val   Loss: {val_loss:.4f}, F1 Score: {val_f1:.4f}\n") # Update print


            # Check for early stopping based on validation F1 score
            if val_f1 > self.best_f1 + self.min_delta:
                self.best_f1 = val_f1
                self.best_model_wts = copy.deepcopy(self.model.state_dict())
                self.counter = 0 # Reset counter
            else:
                self.counter += 1
                if self.counter >= self.patience:
                    self.early_stop = True
                    print(f"Early stopping triggered after {epoch+1} epochs due to no improvement in validation F1 score for {self.patience} consecutive epochs.")
                    break # Stop training loop

        print(f"Training complete. Best val F1: {self.best_f1:.4f}") # Update print
        self.model.load_state_dict(self.best_model_wts)
        return self.model, self.history

In [36]:
from transformers import AutoFeatureExtractor, CvtForImageClassification
import torch.nn as nn
import torch.optim as optim
import torch # Import torch

# Load a pre-trained Cvt model and its feature extractor
model_name = "microsoft/cvt-13" # Example Cvt model
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
# Load the model with the correct number of labels and ignore mismatched sizes
num_classes = 6 # Assuming 6 classes based on the dataset
cvt_model = CvtForImageClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True)


print(f"Model '{model_name}' and feature extractor loaded.")
print(f"Cvt model's classifier head modified for {num_classes} classes.")

# Define the loss function and optimizer for the Cvt model
criterion_cvt = nn.CrossEntropyLoss()
optimizer_cvt = optim.Adam(cvt_model.parameters(), lr=0.001)

print("Loss function and optimizer defined for Cvt model.")

# Instantiate the Trainer and start training for the Cvt model
# Ensure you have train_dataloader and val_dataloader defined
trainer_cvt = Trainer(cvt_model, criterion_cvt, optimizer_cvt, device='cuda')

print("Starting training for Cvt model...")
trained_cvt_model, history_cvt = trainer_cvt.fit(train_dataloader, val_dataloader, num_epochs=5)

print("\nCvt model training complete.")

/usr/local/lib/python3.12/dist-packages/transformers/models/convnext/feature_extraction_convnext.py:30: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(
Some weights of CvtForImageClassification were not initialized from the model checkpoint at microsoft/cvt-13 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([6, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model 'microsoft/cvt-13' and feature extractor loaded.
Cvt model's classifier head modified for 6 classes.
Loss function and optimizer defined for Cvt model.


/tmp/ipython-input-3445361534.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=mixed_precision)


Starting training for Cvt model...
Epoch 1/10
------------------------------


Epoch 1 [val]: 100%|██████████| 61/61 [00:28<00:00,  2.12it/s]


...

Train Loss: 0.7569, F1 Score: 0.6932
Val   Loss: 0.6939, F1 Score: 0.7477

Epoch 8/10
------------------------------


Epoch 8 [val]: 100%|██████████| 61/61 [00:28<00:00,  2.11it/s]


Train Loss: 0.7325, F1 Score: 0.6996
Val   Loss: 3.0152, F1 Score: 0.3585

Epoch 9/10
------------------------------


Epoch 9 [train]:  15%|█▍        | 36/243 [00:20<02:00,  1.72it/s]


KeyboardInterrupt: 

In [45]:
import torch.nn as nn
import torch.optim as optim
import torch
from torchvision.models import efficientnet_b0, efficientnet_b1, efficientnet_b2, efficientnet_b3, EfficientNet_B0_Weights, EfficientNet_B1_Weights, EfficientNet_B2_Weights, EfficientNet_B3_Weights

# Choose the EfficientNet variant you want to fine-tune (e.g., 'efficientnet_b0')
efficientnet_variant = 'efficientnet_b0'

# Load the specified pre-trained EfficientNet model
if efficientnet_variant == 'efficientnet_b0':
    weights = EfficientNet_B0_Weights.IMAGENET1K_V1
    efficientnet_model = efficientnet_b0(weights=weights)
elif efficientnet_variant == 'efficientnet_b1':
    weights = EfficientNet_B1_Weights.IMAGENET1K_V1
    efficientnet_model = efficientnet_b1(weights=weights)
elif efficientnet_variant == 'efficientnet_b2':
    weights = EfficientNet_B2_Weights.IMAGENET1K_V1
    efficientnet_model = efficientnet_b2(weights=weights)
elif efficientnet_variant == 'efficientnet_b3':
    weights = EfficientNet_B3_Weights.IMAGENET1K_V1
    efficientnet_model = efficientnet_b3(weights=weights)
else:
    raise ValueError(f"Unsupported EfficientNet variant: {efficientnet_variant}")

print(f"Pre-trained {efficientnet_variant} model loaded.")

# Modify the classifier head for the number of classes in your dataset
num_classes = 6 # Assuming 6 classes based on your dataset
num_ftrs = efficientnet_model.classifier[1].in_features
efficientnet_model.classifier[1] = nn.Linear(num_ftrs, num_classes)

print(f"{efficientnet_variant} model's classification head modified for {num_classes} classes.")

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficientnet_model.parameters(), lr=0.001)

print("Loss function and optimizer defined.")

# Instantiate the Trainer and start fine-tuning
# Ensure train_dataloader and val_dataloader are defined
trainer = Trainer(efficientnet_model, criterion, optimizer, device='cuda')

print(f"Starting fine-tuning for {efficientnet_variant} model...")
trained_efficientnet_model, history = trainer.fit(train_dataloader, val_dataloader, num_epochs=10)

print(f"\n{efficientnet_variant} fine-tuning complete.")

/tmp/ipython-input-3445361534.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=mixed_precision)


Pre-trained efficientnet_b0 model loaded.
efficientnet_b0 model's classification head modified for 6 classes.
Loss function and optimizer defined.
Starting fine-tuning for efficientnet_b0 model...
Epoch 1/10
------------------------------


Epoch 1 [val]: 100%|██████████| 61/61 [00:30<00:00,  1.99it/s]


Train Loss: 0.4701, F1 Score: 0.8385
Val   Loss: 0.2111, F1 Score: 0.9274

Epoch 2/10
------------------------------


Epoch 2 [val]: 100%|██████████| 61/61 [00:26<00:00,  2.28it/s]


...

Epoch 5 [val]: 100%|██████████| 61/61 [00:26<00:00,  2.32it/s]


Train Loss: 0.0906, F1 Score: 0.9711
Val   Loss: 0.1195, F1 Score: 0.9590

Epoch 6/10
------------------------------


Epoch 6 [train]:   4%|▍         | 10/243 [00:05<01:58,  1.97it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2941639309.py", line 45, in <cell line: 0>
    trained_efficientnet_model, history = trainer.fit(train_dataloader, val_dataloader, num_epochs=10)
                                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-3445361534.py", line 77, in fit
    train_loss, train_f1 = self._run_epoch(train_loader, phase='train') # Get train loss and F1
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-3445361534.py", line 40, in _run_epoch
    for inputs, labels in tqdm(dataloader, desc=f"Epoch {self.epoch+1} [{phase}]"):
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/tqdm/std.py", line 11

TypeError: object of type 'NoneType' has no len()

# Task
Generate Python code to perform inference using a trained model on a test dataset and create a `submission.csv` file in the format "image_id,label".

## Load the trained model

### Subtask:
Load the best performing trained model (e.g., EfficientNet or CvT).


**Reasoning**:
Load the state dictionary of the best performing model and set it to evaluation mode.



In [46]:
efficientnet_model.load_state_dict(trainer.best_model_wts)
efficientnet_model.eval()
print("Best performing model loaded and set to evaluation mode.")

Best performing model loaded and set to evaluation mode.


In [50]:
# Load the best performing trained model weights
# Assuming 'trainer' and 'efficientnet_model' (or your chosen model) are available from training
efficientnet_model.load_state_dict(trainer.best_model_wts)
efficientnet_model.eval() # Set the model to evaluation mode

print("Best performing model loaded and set to evaluation mode.")

Best performing model loaded and set to evaluation mode.


In [51]:
# Prepare test data
# Assuming test images are in a directory named 'test_images' within the competition path
test_img_dir = os.path.join(aikuthon9th_path, 'test_images')

# Create a list of test image file paths
test_image_files = [os.path.join(test_img_dir, f) for f in os.listdir(test_img_dir) if f.endswith('.png')]
test_image_files.sort() # Ensure consistent order

# Create a DataFrame for test images (just with image names/ids)
# We need to extract the image ID from the filename (e.g., "00001.png" -> 1)
test_ids = [int(os.path.splitext(os.path.basename(f))[0]) for f in test_image_files]
test_df = pd.DataFrame({'id': test_ids, 'filename': test_image_files})

# Define a test dataset class (without labels)
class AikuthonTestDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 1] # Get filename
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        # Return image and its original id for submission
        return image, self.dataframe.iloc[idx, 0] # Return image and id

# Use the same transform as training/validation or adjust if needed
# Assuming 'transform' from training is suitable for test data
test_dataset = AikuthonTestDataset(test_df, transform=transform)

# Create a DataLoader for the test dataset
# Use a batch size that fits in memory, shuffle should be False for consistent output order
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Test data prepared and DataLoader created.")
print("First 5 test images in DataFrame:")
display(test_df.head())

Test data prepared and DataLoader created.
First 5 test images in DataFrame:


,id,filename
0,1,/root/.cache/kagglehub/competitions/aikuthon9t...
1,2,/root/.cache/kagglehub/competitions/aikuthon9t...
2,3,/root/.cache/kagglehub/competitions/aikuthon9t...
3,4,/root/.cache/kagglehub/competitions/aikuthon9t...
4,5,/root/.cache/kagglehub/competitions/aikuthon9t...


In [53]:
import torch
from tqdm import tqdm # Import tqdm

# Assuming efficientnet_model (or your trained model) and test_dataloader are defined

# Perform inference on the test data
predictions = []
image_ids = []

# Set the model to evaluation mode (already done in step 1, but good practice to ensure)
efficientnet_model.eval()

# Get the model's device
device = next(efficientnet_model.parameters()).device
print(f"Using device: {device}")


# Disable gradient calculation for inference
with torch.no_grad():
    # Iterate through the test dataloader
    for images, ids in tqdm(test_dataloader, desc="Performing Inference"):
        # Move images to the same device as the model
        images = images.to(device) # Use the obtained device


        # Get model outputs (logits)
        outputs = efficientnet_model(images)

        # Get predicted class indices
        _, predicted_labels = torch.max(outputs, 1)

        # Append predictions and image ids to lists
        predictions.extend(predicted_labels.cpu().numpy())
        image_ids.extend(ids.cpu().numpy())

print("\nInference complete. Collected predictions and image IDs.")
print(f"Number of predictions: {len(predictions)}")
print(f"Number of image IDs: {len(image_ids)}")

Using device: cuda:0


Performing Inference: 100%|██████████| 47/47 [00:23<00:00,  1.96it/s]


Inference complete. Collected predictions and image IDs.
Number of predictions: 1500
Number of image IDs: 1500


In [54]:
import pandas as pd

# Assuming image_ids and predictions lists are available from the inference step

# Create a DataFrame for the submission file
submission_df = pd.DataFrame({'id': image_ids, 'label': predictions})

# Sort by image id to ensure correct order
submission_df = submission_df.sort_values(by='id')

# Generate the submission CSV file
submission_file_path = 'submission.csv'
submission_df.to_csv(submission_file_path, index=False)

print(f"Submission file '{submission_file_path}' created successfully.")
display(submission_df.head())

Submission file 'submission.csv' created successfully.


,id,label
0,1,4
1,2,3
2,3,1
3,4,1
4,5,1


In [55]:
from google.colab import files

# Assuming the submission file is named 'submission.csv'
submission_file_path = 'submission.csv'

try:
    files.download(submission_file_path)
    print(f"'{submission_file_path}' 파일 다운로드를 요청했습니다.")
except FileNotFoundError:
    print(f"오류: '{submission_file_path}' 파일을 찾을 수 없습니다. 파일이 올바르게 생성되었는지 확인하세요.")
except Exception as e:
    print(f"파일 다운로드 중 오류가 발생했습니다: {e}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'submission.csv' 파일 다운로드를 요청했습니다.
